In [39]:
import math
import numpy as np
import matplotlib as plt
from scipy import stats
import pandas as pd
import sys
sys.path.append('../tdiff_path/')

In [40]:
# import some modules that I created to do some data processing.

import dataset_operations.dataset_operations as do 
import retrieve_data.retrieve_data as retrieve

In [52]:
site_file_metadata=pd.read_csv('site_file_metadata.csv')

print(site_file_metadata)

   site      date  interim_data  \
0   SAS  20170627         False   
1   PGR  20171003         False   
2   PGR  20170930          True   
3   CLY  20160628          True   
4   CLY  20160701         False   
5   INV  20170811         False   
6   RKN  20160705          True   
7   RKN  20160721         False   
8   SAS  20170630         False   
9   PGR  20170930         False   
10  CLY  20160701         False   
11  INV  20170813         False   
12  RKN  20160719         False   
13  SAS  20170629         False   
14  PGR  20171001         False   
15  CLY  20160701         False   
16  INV  20170813         False   
17  RKN  20160719         False   
18  RKN  20180808          True   
19  RKN  20180808         False   
20  RKN  20180808         False   
21  RKN  20180808          True   
22  RKN  20180808         False   
23  CLY  20180817          True   
24  CLY  20180817         False   
25  CLY  20180817         False   
26  CLY  20180817          True   
27  CLY  20180817   

In [53]:
# For feedline paths, we need to find out the cable model so that we
#  can properly analyze the data and determine antenna differences.

feedline_metadata = pd.read_csv('site_feedline_metadata.csv')

print(feedline_metadata)

   site array  feedline_number  cable_type  cable_length_ft  \
0   SAS  main                0  Belden8237              600   
1   SAS  main                1  Belden8237              600   
2   SAS  main                2  Belden8237              600   
3   SAS  main                3  Belden8237              600   
4   SAS  main                4  Belden8237              600   
5   SAS  main                5  Belden8237              600   
6   SAS  main                6  Belden8237              600   
7   SAS  main                7  Belden8237              600   
8   SAS  main                8  Belden8237              600   
9   SAS  main                9  Belden8237              600   
10  SAS  main               10  Belden8237              600   
11  SAS  main               11  Belden8237              600   
12  SAS  main               12  Belden8237              600   
13  SAS  main               13  Belden8237              600   
14  SAS  main               14  Belden8237             

In [65]:
# subsetting for feedline data at Saskatoon
feedline_vswr_data=site_file_metadata[site_file_metadata['data_type']=='feedline-VSWR']
print(feedline_vswr_data)
sas_feedlines=feedline

   site      date  interim_data  \
0   SAS  20170627         False   
1   PGR  20171003         False   
2   PGR  20170930          True   
3   CLY  20160628          True   
4   CLY  20160701         False   
5   INV  20170811         False   
6   RKN  20160705          True   
7   RKN  20160721         False   
18  RKN  20180808          True   
19  RKN  20180808         False   
23  CLY  20180817          True   
24  CLY  20180817         False   

                                        data_location  \
0   /home/shared/Sync/Sites/Saskatoon/SITE-VISITS-...   
1   /home/shared/Sync/Sites/Prince_George/Trips/20...   
2   /home/shared/Sync/Sites/Prince_George/Trips/20...   
3   /home/shared/Sync/Sites/Clyde_River/Trips/2016...   
4   /home/shared/Sync/Sites/Clyde_River/Trips/2016...   
5     /home/shared/Sync/Sites/Inuvik/Trips/2017/DATA/   
6   /home/shared/Sync/Sites/Rankin_Inlet/Trips/201...   
7   /home/shared/Sync/Sites/Rankin_Inlet/Trips/201...   
18  /home/shared/Sync/Sites/Ran